In [30]:
import csv
import requests
import xml.etree.ElementTree as ET
from xml.dom.minidom import parse, parseString


In [31]:
cc_xml = ''
with open('./Conversation_SSML.xml', 'r') as cc_file:
    cc_xml = cc_file.read()

In [32]:
conversation = []
i=1

tree = parseString(cc_xml)
for el in tree.getElementsByTagName('voice'):
    conv_part = ''
    speaker = (dict(el.attributes.items())['name'])
    for node in el.childNodes:
        if node.nodeType == node.TEXT_NODE:
            conv_part = node.data
        elif node.tagName == 'prosody':
            conv_part = conv_part + node.firstChild.data
        else: 
            for pro in node.getElementsByTagName('prosody'):
                conv_part = conv_part + pro.firstChild.data
    part = 'Agent'
    if speaker == 'en-US-JennyNeural' : 
        part = 'Agent'
    else:
        part = 'Customer'
    conversation.append({"text":conv_part, 'id':i, 'role':part, 'participantId':'{}'.format(part)})
    i = i+1
    
#print(conversation)


In [33]:
# Format Conversation for Language Service Analysis

json_d = { "displayName": "Conversation Task Example",
  "analysisInput": {
    "conversations": [
      {
        "conversationItems": conversation ,
        "modality": "text",
        "id": "conversation1",
        "language": "en"
      }
    ]
  },
  "tasks": [             
       {
        "taskName": "analyze 1",
        "kind": "ConversationalSummarizationTask",
        "parameters": {
           "summaryAspects": ["chaptertitle","issue", "resolution", "narrative"]

        }
    }
  ]
}

In [34]:
json_d

{'displayName': 'Conversation Task Example',
 'analysisInput': {'conversations': [{'conversationItems': [{'text': 'Hello, how may I help you today?',
      'id': 1,
      'role': 'Agent',
      'participantId': 'Agent'},
     {'text': "I really need help with my credit card, it's not working at all",
      'id': 2,
      'role': 'Customer',
      'participantId': 'Customer'},
     {'text': 'May I please have your first and last name?',
      'id': 3,
      'role': 'Agent',
      'participantId': 'Agent'},
     {'text': "sure it's John, J O H N, Doh, D O E",
      'id': 4,
      'role': 'Customer',
      'participantId': 'Customer'},
     {'text': 'Thank you Mr Doh, can you confirm the last four digits of your account number?',
      'id': 5,
      'role': 'Agent',
      'participantId': 'Agent'},
     {'text': "Which number? Is that the card number or the number on my statement, I don't have a statement in front of me.",
      'id': 6,
      'role': 'Customer',
      'participantId': '

## Open AI Summarization

In [80]:
import os
import openai
from configparser import ConfigParser

parser=ConfigParser()
_=parser.read('../config.cfg')
openai.api_type = "azure"
openai.api_base = parser.get('openai_api','api_ep')
openai.api_version = "2022-12-01"
openai.api_key = parser.get('openai_api','api_key')
model = parser.get('openai_api','api_model')

In [81]:
prompt = "Generate detailed call summary notes in list format from the following text\nText:\n###\n{}\n###\n- ".format(conversation)

response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.2,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
print(response['choices'][0]['text'])

Response:
 Call Summary Notes:
1. Customer contacted agent for help with credit card not working.
2. Agent requested customer's first and last name, which was provided.
3. Agent requested last four digits of account number, which was provided.
4. Agent informed customer of suspected fraud on credit card and asked for last purchase made.
5. Customer confirmed last purchase was for an Air Bee En Bee stay in December in Florida.
6. Agent confirmed charge amount and marked it as approved.
7. Customer thanked agent and proceeded with booking.
8. Customer requested another card for daughter, Jane Doh.
9. Agent requested address to mail card, which was provided.
10. Agent informed customer card would arrive in 1-2 business days.
11. Customer thanked agent and ended call.


In [27]:
prompt = "Extract list of call participants in list format from the following text\nText:\n###\n{}\n###\n".format(conversation)

response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.5,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
print(response['choices'][0]['text'])

Response:
List of call participants:
- Agent
- Customer


In [29]:
prompt = "Generate list of call follow up tasks from following text\nText:\n###\n{}\n###\n".format(conversation)

response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.7,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
print(response['choices'][0]['text'])

Response:

List of call follow up tasks:
1. Follow up with customer to confirm the last four digits of their account number
2. Follow up with customer to confirm the charge amount of their last purchase
3. Follow up with customer to confirm the address to mail the new card to
4. Follow up with customer to confirm receipt of the new card


In [82]:
JSONprompt = '''You must extract the following information from the phone conversation below:

	1. Call reason (key: reason)
	2. Cause of incident (key: cause)
	3. Name of caller (key: caller)
	4. Account number (key: account_id)
    5. Follow-up Flag(key: followupind)
	6. A short, yet detailed summary (key: summary)

Make sure fields 1 to 4 are answered very short. Field 5 is a boolean flag which indicates whether the caller incident was resolved, 1 if follow up required, 0 if resolved.  Please answer in JSON machine-readable format, using the keys from above. Format the output as JSON object. Pretty print the JSON and make sure that is properly closed at the end.'''

In [83]:
prompt = '{}Conversation:\n###\n{}\n###'.format(JSONprompt,conversation)

In [84]:
response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.2,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
print(response['choices'][0]['text'])

Response:

{
    "reason": "Credit card not working",
    "cause": "Suspected fraud",
    "caller": "John Doh",
    "account_id": "4321",
    "followupind": 0,
    "summary": "John Doh called to report that his credit card was not working. Suspected fraud was detected, but was confirmed by the customer. A new card was ordered for his daughter Jane Doh and will be mailed to the default address on Pine Wood Ave."
}


In [85]:
eval(response['choices'][0]['text'])

{'reason': 'Credit card not working',
 'cause': 'Suspected fraud',
 'caller': 'John Doh',
 'account_id': '4321',
 'followupind': 0,
 'summary': 'John Doh called to report that his credit card was not working. Suspected fraud was detected, but was confirmed by the customer. A new card was ordered for his daughter Jane Doh and will be mailed to the default address on Pine Wood Ave.'}